# Edit Distance
calculate the edit distance between 2 strings

In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
def edit_distance(s1, s2, insertion_penalty=1):
    """
    Compute min # of insertions, deletions, and replacements to transfrom s1 into s2
    """
    # init m x n array
    # first row, column represent blanks
    # moves down represent consuming a char of s1 without emitting to s2
    # moves right represent an insertion into s2 without emitting to s2
    # moves diagonal represent copy (free) or replacement (copy s1's char + mutate into s2's char)
    m = len(s1) + 1
    n = len(s2) + 1
    a = [[0 for j in range(n)] for i in range(m)]
    for i in range(1, m):
        a[i][0] = i
    for j in range(1, n):
        a[0][j] = i
    for i in range(1, m):
        for j in range(1, n):
            copy_or_replace = a[i-1][j-1] if s1[i-1] == s2[j-1] else 1 + a[i-1][j-1]
            insert = insertion_penalty + a[i][j-1]
            delete = 1 + a[i-1][j]
            a[i][j] = min(copy_or_replace, insert, delete)
    return a[-1][-1]

In [3]:
def edit_distance_free_insertion(s1, s2):
    return edit_distance(s1, s2, insertion_penalty=0)

In [4]:
def edit_distance_cheap_insertion(s1, s2):    
    expected_insertions = len(s2) - len(s1)
    total_insertion_penalty = math.log(expected_insertions, 2) if expected_insertions > 0 else 0
    per_char_insertion_penalty = total_insertion_penalty / expected_insertions if expected_insertions > 0 else 0
    return edit_distance(s1, s2, insertion_penalty=per_char_insertion_penalty)

In [5]:
edit_distance('smrtwtr', 'smartwater')

3

In [6]:
def similarity(s1, s2):
    max_distance = max(len(s1), len(s2))
    return 1 - edit_distance(s1, s2) / max_distance

In [7]:
def similarity_free_insertion(s1, s2):
    max_distance = min(len(s1), len(s2))
    return 1 - edit_distance_free_insertion(s1, s2) / max_distance

In [8]:
def similarity_cheap_insertion(s1, s2):
    expected_insertions = max(0, len(s2) - len(s1))
    max_distance = min(len(s1), len(s2)) + (math.log(expected_insertions,2) if expected_insertions > 0 else 0)
    return 1 - edit_distance_cheap_insertion(s1, s2) / max_distance

In [9]:
similarity('smrtwtr', 'smartwater')

0.7

In [10]:
similarity_free_insertion('smrtwtr', 'smartwater')

1.0

In [11]:
similarity_cheap_insertion('smrtwtr', 'smartwater')

0.8153792167888892

In [12]:
similarity_cheap_insertion('smartwater', 'smrtwtr')

0.5714285714285714

In [13]:
def phrase_similarity(p1, p2, similarity=similarity):
    """
    Compute similarity of 2 phrases
    """
    l1 = p1.split()
    l2 = p2.split()
    shorter = l1 if len(l1) < len(l2) else l2
    longer = l2 if l1 == shorter else l1
    shorter.sort(key=lambda x: -len(x))
    res = []
    res2 = []
    for i, w1 in enumerate(shorter):
        scores = [w1] + [(w2, similarity(w1, w2)) for w2 in longer]
        res.append(scores)
    for w1 in shorter:
        scores = [similarity(w1, w2) for w2 in longer]
        max_score = max(scores)
        max_score_idx = np.array(scores).argmax()
        res2.append((max_score, w1, longer[max_score_idx]))
        if max_score:
            del longer[max_score_idx]
            
    avg_score = np.average(np.array([score for score, _, _ in res2], dtype=float))

#     return res, res2, avg_score
    return avg_score

In [14]:
phrase_similarity('SFTSOAP KTCHN FRSH', 'Softsoap Fresh Breeze Liquid Hand Soap'.upper())

0.5583333333333333

In [15]:
phrase_similarity('SFTSOAP KTCHN FRSH', 'Softsoap Fresh Breeze Liquid Hand Soap'.upper(), similarity=similarity_free_insertion)

0.75

In [16]:
phrase_similarity('SFTSOAP KTCHN FRSH', 'Softsoap Fresh Breeze Liquid Hand Soap'.upper(), similarity=similarity_cheap_insertion)

0.75

In [17]:
def matches(short_phrase, df, similarity_fn=similarity):
    df2 = df.copy()
    df2['score'] = df2.apply(lambda x: phrase_similarity(short_phrase, x['upper_web'], similarity=similarity_fn), axis=1)
    df2.sort_values('score', ascending=False, inplace=True)
    return df2

In [18]:
def match_1(short_phrase, df, similarity_fn=similarity):
    df2 = matches(short_phrase, df, similarity_fn=similarity_fn)
    return df2.iloc[0]['web']

In [19]:
def match_2(short_phrase, df, similarity_fn=similarity):
    df2 = matches(short_phrase, df, similarity_fn=similarity_fn)
    return df2.iloc[1]['web']

In [20]:
def match_3(short_phrase, df, similarity_fn=similarity):
    df2 = matches(short_phrase, df, similarity_fn=similarity_fn)
    return df2.iloc[2]['web']

In [21]:
def find_matches(df, similarity_fn=similarity):
    df['upper_web'] = df.apply(lambda x: x['web'].upper() ,axis=1)
    df['match_1'] = df.apply(lambda x: match_1(x['raw'], df, similarity_fn=similarity_fn), axis=1)
    df['match_2'] = df.apply(lambda x: match_2(x['raw'], df, similarity_fn=similarity_fn), axis=1)
    df['match_3'] = df.apply(lambda x: match_3(x['raw'], df, similarity_fn=similarity_fn), axis=1)
    df['correct_1'] = df.apply(lambda x: x['match_1'] == x['web'], axis=1)
    df['correct_2'] = df.apply(lambda x: x['match_2'] == x['web'], axis=1)  
    df['correct_3'] = df.apply(lambda x: x['match_3'] == x['web'], axis=1)    
    df['correct_any'] = df.apply(lambda x: x['correct_1'] or x['correct_2'] or x['correct_3'], axis=1)        

In [22]:
phrase_similarity('ASP ORG', 'ORGANIC - ASPARAGUS')

0.4365079365079365

In [23]:
phrase_similarity('ASP ORG', 'ORGANIC - ASPARAGUS', similarity=similarity_free_insertion)

1.0

In [24]:
phrase_similarity('ASP ORG', 'ORGANIC - ASPARAGUS', similarity=similarity_cheap_insertion)

0.568578347626562

In [25]:
phrase_similarity('ASP ORG', 'Del Cabo Cucumber Og 16 Oz'.upper())

0.45833333333333337

In [26]:
phrase_similarity('ASP ORG', 'Del Cabo Cucumber Og 16 Oz'.upper(), similarity=similarity_free_insertion)

0.25

In [27]:
phrase_similarity('ASP ORG', 'Del Cabo Cucumber Og 16 Oz'.upper(), similarity_cheap_insertion)

0.25

In [28]:
def precision_at_1(df):
    s = df['correct_1'].value_counts()
    return s.loc[True] / (s.loc[True] + s.loc[False])

In [29]:
def precision_at_3(df):
    s = df['correct_any'].value_counts()
    return s.loc[True] / (s.loc[True] + s.loc[False])

In [30]:
%alias_magic t timeit

Created `%t` as an alias for `%timeit`.
Created `%%t` as an alias for `%%timeit`.


In [31]:
%t -n1 edit_distance('smrtwtr', 'smartwater')

67.1 µs ± 5.71 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
def experiment(similarity_fn=similarity):
    path = '../data/raw_web_joined/703_00198_2020-03-20_3_1391204_joined.json'
    df = pd.read_json(path)
    find_matches(df, similarity_fn=similarity_fn)
    p1 = precision_at_1(df)
    p3 = precision_at_3(df)
    stats = pd.Series({'p_1': p1, 'p_3': p3})
    return stats

In [33]:
e1 = experiment()
e1

p_1    0.702381
p_3    0.892857
dtype: float64

In [34]:
e2 = experiment(similarity_fn=similarity_free_insertion)
e2

p_1    0.630952
p_3    0.857143
dtype: float64

In [35]:
e3 = experiment(similarity_fn=similarity_cheap_insertion)
e3

p_1    0.702381
p_3    0.880952
dtype: float64

In [37]:
# TODO
# why doesn't free/cheap insertions work well?  See ASP ORG example above
# maybe memoize similarity